# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import gradio as gr
import google.generativeai as genai
from google.generativeai.protos import FunctionResponse, Part
import os
import json
from dotenv import load_dotenv

In [2]:
load_dotenv()
# --- Configuration and Setup ---

# --- IMPORTANT: Configure the Gemini API Key ---
# It's best practice to set this as an environment variable
try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY environment variable not set.")
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    print(f"Error configuring Gemini API: {e}")
    # Exit if the API key isn't configured, as the app cannot run.
    exit()


In [3]:
# The absolute path to the root directory the LLM is allowed to access.
ALLOWED_DIRECTORY = os.path.expanduser("~/projects/pleasurewebsite/mysite")

if not os.path.isdir(ALLOWED_DIRECTORY):
    print(f"Error: The specified directory '{ALLOWED_DIRECTORY}' does not exist.")
    exit()

# --- Tool Functions (with type hints for better schema generation) ---

def list_files_in_directory(path: str = "."):
    """
    Lists files and directories within a specified path inside the allowed directory.
    The path is relative to the project root.
    """
    try:
        base_path = ALLOWED_DIRECTORY
        requested_path = os.path.abspath(os.path.join(base_path, path))
        if not requested_path.startswith(base_path):
            return {"error": "Access denied."}
        if not os.path.isdir(requested_path):
            return {"error": f"'{path}' is not a valid directory."}
        items = os.listdir(requested_path)
        dirs = sorted([d for d in items if os.path.isdir(os.path.join(requested_path, d))])
        files = sorted([f for f in items if os.path.isfile(os.path.join(requested_path, f))])
        
        response_str = f"Contents of '{path}':\n\n"
        if dirs:
            response_str += "Directories:\n" + "\n".join(f"- {d}/" for d in dirs) + "\n\n"
        if files:
            response_str += "Files:\n" + "\n".join(f"- {f}" for f in files) + "\n"
        
        return {"content": response_str if items else f"The directory '{path}' is empty."}
    except Exception as e:
        return {"error": f"An error occurred: {str(e)}"}


In [4]:
def read_file_content(file_path: str):
    """
    Reads the text content of a specific file within the allowed directory.
    The file_path is relative to the project root.
    """
    try:
        base_path = ALLOWED_DIRECTORY
        requested_path = os.path.abspath(os.path.join(base_path, file_path))
        if not requested_path.startswith(base_path):
            return {"error": "Access denied."}
        if not os.path.isfile(requested_path):
            return {"error": f"File not found at '{file_path}'"}
        with open(requested_path, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read(5000)
            final_content = content + "\n\n[... file content truncated ...]" if len(content) == 5000 else content
            return {"content": final_content}
    except Exception as e:
        return {"error": f"An error occurred while reading the file: {str(e)}"}


In [5]:
# --- Tool Definitions for the OpenAI API ---

tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files_in_directory",
            "description": "Get a list of files and directories in a specified path within the Django project. Use '.' for the root.",
            "parameters": { "type": "object", "properties": { "path": { "type": "string", "description": "The relative path to the directory. E.g., 'home/views'."}}, "required": ["path"]},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "read_file_content",
            "description": "Read the text content of a specific file within the Django project.",
            "parameters": { "type": "object", "properties": { "file_path": { "type": "string", "description": "The relative path to the file. E.g., 'mysite/settings.py'."}}, "required": ["file_path"]},
        },
    },
]


In [6]:
# --- Core Chat Logic (Rewritten for Gemini) ---

def chat_with_tools(user_message, history):
    """
    Handles the conversation using Google Gemini, including tool calls and streaming.
    """
    system_instruction = f"You are a helpful AI assistant with access to a local Django project file system. The root of the project is '{ALLOWED_DIRECTORY}'. You can list files and read their content. When asked about the project, use your tools to find information before answering. Be concise."
    
    # Pass the Python functions directly to the model for automatic schema generation.
    model = genai.GenerativeModel(
        model_name='gemini-1.5-flash',
        system_instruction=system_instruction,
        tools=[list_files_in_directory, read_file_content],
    )

    # A dictionary to map function names (strings) to the actual functions
    available_functions = {
        "list_files_in_directory": list_files_in_directory,
        "read_file_content": read_file_content,
    }

    # Convert Gradio history to Gemini's message format
    messages = []
    for human_msg, ai_msg in history:
        messages.append({"role": "user", "parts": [human_msg]})
        messages.append({"role": "model", "parts": [ai_msg]})
    
    # Start a chat session to maintain conversation history
    chat_session = model.start_chat(history=messages)

    try:
        # Send the message to Gemini (NON-STREAMING to check for tool calls)
        response = chat_session.send_message(user_message)
        
        # --- THIS IS THE FIX ---
        # Safely check for a function call in the response parts
        function_call = None
        # Ensure the response has candidates and parts before trying to access them
        if response.candidates and response.candidates[0].content.parts:
            part = response.candidates[0].content.parts[0]
            # Check if the part contains a function_call and that it has a name
            if 'function_call' in part and part.function_call.name:
                function_call = part.function_call

        if function_call:
            # The model wants to use a tool
            function_name = function_call.name
            function_args = {key: value for key, value in function_call.args.items()}
            
            print(f"🤖 Calling tool: {function_name}({function_args})")
            
            # Call the actual Python function
            function_response_data = available_functions[function_name](**function_args)
            
            # Send the tool's output back to the model for the final answer
            response_stream = chat_session.send_message(
                Part(
                    function_response=FunctionResponse(
                        name=function_name,
                        response=function_response_data,
                    )
                ),
                stream=True,
            )
            
            for chunk in response_stream:
                if chunk.text:
                    yield chunk.text

        else:
            # No tool call, the model responded directly. Stream this response.
            # Safely check for text content before yielding it.
            if response.candidates and response.candidates[0].content.parts and response.candidates[0].content.parts[0].text:
                 yield response.text
            else:
                 yield "The model did not provide a text response or a tool call."

    except Exception as e:
        print(f"An error occurred in the Gemini chat logic: {e}")
        yield "Sorry, an error occurred with the AI model. Please check the console for details."


In [7]:

# --- Gradio Interface (This part remains the same) ---

with gr.Blocks(theme="soft", title="Django Project Assistant (Gemini) 🤖") as demo:
    gr.Markdown("# Django Project Assistant (with Gemini) 🤖")
    gr.Markdown("Ask me questions about your Django project. I can list files and read their contents to help you.")
    
    chatbot = gr.Chatbot(label="Conversation", height=600, bubble_full_width=False) 
    
    with gr.Row():
        msg_textbox = gr.Textbox(
            label="Your Message",
            placeholder="e.g., What files are in mysite/?",
            scale=7,
            autofocus=True,
        )
        submit_btn = gr.Button("Send", variant="primary", scale=1)

    def handle_chat(user_input, history):
        history.append([user_input, ""])
        response_stream = chat_with_tools(user_input, history[:-1])
        for chunk in response_stream:
            history[-1][1] += chunk
            yield history, ""
        yield history, ""

    submit_btn.click(handle_chat, [msg_textbox, chatbot], [chatbot, msg_textbox])
    msg_textbox.submit(handle_chat, [msg_textbox, chatbot], [chatbot, msg_textbox])

    gr.Examples(
        examples=[
            ["What files are in the root directory?"],
            ["Can you show me the contents of 'mysite/settings.py'?"],
            ["What's inside the 'home/templates/home' directory?"]
        ],
        inputs=msg_textbox,
        outputs=[chatbot, msg_textbox],
        fn=handle_chat,
        cache_examples=False,
    )



/tmp/ipykernel_336180/2592309743.py:7: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation", height=600, bubble_full_width=False)
/tmp/ipykernel_336180/2592309743.py:7: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(label="Conversation", height=600, bubble_full_width=False)


In [ ]:
if __name__ == "__main__":
    demo.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


🤖 Calling tool: list_files_in_directory({'path': '/home/pradeep/projects/pleasurewebsite/mysite'})
🤖 Calling tool: read_file_content({'file_path': 'mysite/settings.py'})
🤖 Calling tool: list_files_in_directory({'path': 'home/templates/home'})
